In [2]:
import pandas as pd
from qiskit import transpile
from tabulate import tabulate
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit_aer import AerSimulator
from numpy import pi

In [3]:
theta = 1.04720 # 🐓считать по варианту 🐔

def get_df_combined(circuit, shots):
    simulator = AerSimulator()
    compiled_circuit = transpile(circuit, simulator)
    result = simulator.run(compiled_circuit, shots=shots).result()

    counts = result.get_counts()

    data = {
        'Shots': shots,
        '|1⟩': counts.get('1', 0),
        '|0⟩': counts.get('0', 0),
    }

    data_freq = {
        'Shots': shots,
        '|1⟩': data['|1⟩'] / shots,
        '|0⟩': data['|0⟩'] / shots,
    }

    df_quantity = pd.DataFrame([data], columns=['Shots', '|1⟩', '|0⟩'])
    df_frequency = pd.DataFrame([data_freq], columns=['Shots', '|1⟩', '|0⟩'])

    df_combined = pd.concat([df_quantity, df_frequency.iloc[:, 1:]], axis=1)
    df_combined.columns = pd.MultiIndex.from_tuples([
        ('', 'Shots'),
        ('Frequency (quantity)', '|1⟩'),
        ('Frequency (quantity)', '|0⟩'),
        ('Frequency (out of 1)', '|1⟩'),
        ('Frequency (out of 1)', '|0⟩'),
    ])

    df_combined.columns = [' '.join(col).strip() for col in df_combined.columns]
    
    return df_combined
    

#### 6.1. Получить кубит в состоянии суперпозиции ZALUPA

In [4]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.h(qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 1024
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 1024.0 |          513.0           |          511.0           |       0.5009765625       |       0.4990234375       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.2 Двумя способами получите кубит в состоянии суперпозиции ZALUPA

In [5]:
# FIRST

qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.x(qreg_q[0])
circuit.h(qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 1024
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 1024.0 |          526.0           |          498.0           |       0.513671875        |       0.486328125        |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


In [6]:
# SECOND

qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.h(qreg_q[0])
circuit.z(qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 1024
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 1024.0 |          512.0           |          512.0           |           0.5            |           0.5            |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.3 Получите кубит в состоянии суперпозиции ZALUPA

In [7]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.x(qreg_q[0])
circuit.h(qreg_q[0])
circuit.x(qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 1024
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 1024.0 |          511.0           |          513.0           |       0.4990234375       |       0.5009765625       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.4 С помощью вентиля RX создайте кубит в состоянии ...

In [32]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.rx(theta, qreg_q[0])
circuit.p(pi / 2, qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          521.0           |          1527.0          |      0.25439453125       |      0.74560546875       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.5 С помощью однокубитного вентиля RY получите кубит в состоянии суперпозиции ...

In [9]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.ry(theta, qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk

circuit.measure(qreg_q[0], creg_c[0])

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          498.0           |          1550.0          |       0.2431640625       |       0.7568359375       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.6 С помощью однокубитного вентиля U получите кубит в состоянии суперпозиции 

In [10]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.u(theta, 0, 0, qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          532.0           |          1516.0          |       0.259765625        |       0.740234375        |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.7 С помощью однокубитного вентиля RX получите кубит в состоянии суперпозиции

In [11]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.rx(theta, qreg_q[0])
circuit.p(pi / 2, qreg_q[0])
circuit.z(qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

simulator = AerSimulator()

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          509.0           |          1539.0          |      0.24853515625       |      0.75146484375       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.8 С помощью однокубитного вентиля RY получите кубит в состоянии суперпозиции

In [12]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.ry(theta, qreg_q[0])
circuit.z(qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          517.0           |          1531.0          |      0.25244140625       |      0.74755859375       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.9 С помощью однокубитного вентиля U получите кубит в состоянии суперпозиции ...

In [13]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.u(theta, 0, 0, qreg_q[0])
circuit.z(qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          487.0           |          1561.0          |      0.23779296875       |      0.76220703125       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.10  С помощью вентилей поворота получите кубит в состоянии
##### за это хз

In [14]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.ry(theta, qreg_q[0])
circuit.rz(pi, qreg_q[0])    # за это хз
circuit.rz(pi, qreg_q[0])    # не ебу что они по тз хотели получить
circuit.barrier(qreg_q)      # по факту это получилось +- тоже-самое, как и в 6.5
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          500.0           |          1548.0          |       0.244140625        |       0.755859375        |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.11 С помощью вентиля RX получите кубит в состоянии суперпозиции =>

In [15]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.rx(theta, qreg_q[0])
circuit.rz(pi / 2, qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          531.0           |          1517.0          |      0.25927734375       |      0.74072265625       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


##### => Далее составьте схему, представленную на рис. 20

In [16]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.rx(0.927, qreg_q[0])
circuit.h(qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          1004.0          |          1044.0          |       0.490234375        |       0.509765625        |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.12. С помощью вентиля Rx получите кубит в состоянии суперпозиции =>

In [17]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.reset(qreg_q[0])
circuit.rx(theta, qreg_q[0])
circuit.h(qreg_q[0])
circuit.h(qreg_q[0])
circuit.rz(pi / 2, qreg_q[0])
circuit.barrier(qreg_q)
# @phaseDisk
circuit.measure(qreg_q[0], creg_c[0])

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          509.0           |          1539.0          |      0.24853515625       |      0.75146484375       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


##### => Далее составьте схему, представленную на рис. 21

In [18]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.rx(0.927, qreg_q[0])
circuit.h(qreg_q[0])
circuit.h(qreg_q[0])
circuit.measure(qreg_q[0], creg_c[0])
circuit.barrier(qreg_q)
# @phaseDisk

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          397.0           |          1651.0          |      0.19384765625       |      0.80615234375       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.13. Соберите квантовые схемы показанные на рис. 22.

##### A

In [19]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.h(qreg_q[0])
circuit.measure(qreg_q[0], creg_c[0])
circuit.barrier(qreg_q)
# @phaseDisk

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          990.0           |          1058.0          |       0.4833984375       |       0.5166015625       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


##### B

In [20]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.x(qreg_q[0])
circuit.h(qreg_q[0])
circuit.measure(qreg_q[0], creg_c[0])
circuit.barrier(qreg_q)
# @phaseDisk

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          1011.0          |          1037.0          |      0.49365234375       |      0.50634765625       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


##### C

In [21]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.h(qreg_q[0])
circuit.z(qreg_q[0])
circuit.measure(qreg_q[0], creg_c[0])
circuit.barrier(qreg_q)
# @phaseDisk

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          1033.0          |          1015.0          |      0.50439453125       |      0.49560546875       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


#### 6.14. Соберите квантовые схемы показанные на рис. 23

##### A

In [22]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.h(qreg_q[0])
circuit.h(qreg_q[1])
circuit.measure(qreg_q[0], creg_c[0])
circuit.measure(qreg_q[1], creg_c[0])
circuit.barrier(qreg_q)
# @phaseDisk

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          1012.0          |          1036.0          |       0.494140625        |       0.505859375        |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+


##### B

In [23]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.h(qreg_q[0])
circuit.x(qreg_q[1])
circuit.h(qreg_q[1])
circuit.measure(qreg_q[0], creg_c[0])
circuit.measure(qreg_q[1], creg_c[0])
circuit.barrier(qreg_q)
# @phaseDisk

shots = 2048
df_combined = get_df_combined(circuit, shots)

print(tabulate(df_combined, headers='keys', tablefmt='pretty', stralign='center'))

+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
|   | Shots  | Frequency (quantity) |1⟩ | Frequency (quantity) |0⟩ | Frequency (out of 1) |1⟩ | Frequency (out of 1) |0⟩ |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
| 0 | 2048.0 |          982.0           |          1066.0          |       0.4794921875       |       0.5205078125       |
+---+--------+--------------------------+--------------------------+--------------------------+--------------------------+
